In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [1]:
from keras.layers import Input, Lambda, Dense, Flatten
from keras.models import Model
# from keras.applications.vgg16 import VGG16
from keras.applications.mobilenet_v2 import MobileNetV2
from keras.applications.vgg16 import preprocess_input
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
import numpy as np
from glob import glob
import matplotlib.pyplot as plt
import os

2024-02-14 12:13:54.283176: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-02-14 12:13:54.283288: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-02-14 12:13:54.411203: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [2]:
img_size = [224, 224]
# vgg = VGG16(input_shape = img_size + [3] , weights = 'imagenet' , include_top = False) 
mobile = MobileNetV2(input_shape = img_size + [3] , weights = 'imagenet' , include_top = False)

9406464/9406464 [==============================] - 1s 0us/step


In [3]:
train_path = "/kaggle/input/fruits/fruits-360_dataset/fruits-360/Training"
test_path = "/kaggle/input/fruits/fruits-360_dataset/fruits-360/Test"

In [4]:
# we are not training the existing weights in VGG16 layers
for layer in mobile.layers :
    layer.trainable = False

In [5]:
folders = []
for i in os.listdir(train_path):
    folders.append(train_path + '/' + i)
len(folders)

131

In [6]:
x = Flatten()(mobile.output)
prediction = Dense(len(folders) , activation = 'softmax')(x)
model = Model(inputs = mobile.input , outputs = prediction)
model.summary()

# x = Flatten()(vgg.output)
# prediction = Dense(131 , activation = 'softmax')(x)
# model = Model(inputs = vgg.input, outputs = prediction)
# model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 224, 224, 3)]        0         []                            
                                                                                                  
 Conv1 (Conv2D)              (None, 112, 112, 32)         864       ['input_1[0][0]']             
                                                                                                  
 bn_Conv1 (BatchNormalizati  (None, 112, 112, 32)         128       ['Conv1[0][0]']               
 on)                                                                                              
                                                                                                  
 Conv1_relu (ReLU)           (None, 112, 112, 32)         0         ['bn_Conv1[0][0]']        

In [7]:
model.compile(loss = 'categorical_crossentropy' , optimizer = 'adam' , metrics = ['accuracy'])

In [10]:
from keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(rescale = 1 / 255.0 , shear_range = 0.2 , zoom_range = 0.2 , horizontal_flip = True)
test_datagen = ImageDataGenerator(rescale = 1 / 255.0)
train_set = train_datagen.flow_from_directory(train_path , target_size = (224, 224) , batch_size = 64 , class_mode = 'categorical')
test_set = test_datagen.flow_from_directory(test_path , target_size = (224,224) , batch_size = 1 , class_mode = 'categorical')
print(len(train_set))
print(len(test_set))

Found 67692 images belonging to 131 classes.
Found 22688 images belonging to 131 classes.
1058
22688


In [11]:
r = model.fit_generator(train_set , validation_data = test_set , epochs = 5 , steps_per_epoch = len(train_set) , validation_steps = len(test_set))

/tmp/ipykernel_34/1683211016.py:1: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  r = model.fit_generator(train_set , validation_data = test_set , epochs = 5 , steps_per_epoch = len(train_set) , validation_steps = len(test_set))


Epoch 1/5
   1/1058 [..............................] - ETA: 1:51:15 - loss: 6.0399 - accuracy: 0.0000e+00

I0000 00:00:1707912954.243370      96 device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


1058/1058 [==============================] - 1425s 1s/step - loss: 1.1579 - accuracy: 0.9305 - val_loss: 2.6809 - val_accuracy: 0.9068
Epoch 2/5
1058/1058 [==============================] - 901s 852ms/step - loss: 0.3230 - accuracy: 0.9832 - val_loss: 2.4568 - val_accuracy: 0.9306
Epoch 3/5
1058/1058 [==============================] - 903s 854ms/step - loss: 0.3745 - accuracy: 0.9851 - val_loss: 3.7354 - val_accuracy: 0.9215
Epoch 4/5
1058/1058 [==============================] - 886s 838ms/step - loss: 0.2660 - accuracy: 0.9905 - val_loss: 3.1930 - val_accuracy: 0.9328
Epoch 5/5
1058/1058 [==============================] - 904s 854ms/step - loss: 0.2299 - accuracy: 0.9924 - val_loss: 3.8158 - val_accuracy: 0.9314


In [ ]:
for folder in os.listdir(test_path) :
    cur_path = test_path + "/" + folder
    for img in os.listdir(cur_path) :
        img = image.load_img(path = cur_path + "/" + img , target_size = (224, 224))
        img_array = image.img_to_array(img)
        img_array = np.expand_dims(img_array, axis = 0)
        img_array = preprocess_input(img_array)
        preds = model.predict(img_array)
        label = np.argmax(preds[0])
        print(f'Predicted label: {label}')

1/1 [==============================] - 1s 796ms/step
Predicted label: 57
1/1 [==============================] - 0s 21ms/step
Predicted label: 57
1/1 [==============================] - 0s 21ms/step
Predicted label: 57
1/1 [==============================] - 0s 22ms/step
Predicted label: 57
1/1 [==============================] - 0s 21ms/step
Predicted label: 57
1/1 [==============================] - 0s 21ms/step
Predicted label: 57
1/1 [==============================] - 0s 21ms/step
Predicted label: 57
1/1 [==============================] - 0s 21ms/step
Predicted label: 57
1/1 [==============================] - 0s 22ms/step
Predicted label: 57
1/1 [==============================] - 0s 22ms/step
Predicted label: 57
1/1 [==============================] - 0s 21ms/step
Predicted label: 57
1/1 [==============================] - 0s 23ms/step
Predicted label: 57
1/1 [==============================] - 0s 21ms/step
Predicted label: 57
1/1 [==============================] - 0s 22ms/step
Predicted l

In [14]:
test_loss, test_acc = model.evaluate(test_set, steps=len(test_set), verbose=1)

print(f'Test loss: {test_loss}')
print(f'Test accuracy: {test_acc}')

22688/22688 [==============================] - 156s 7ms/step - loss: 3.8158 - accuracy: 0.9314
Test loss: 3.815781354904175
Test accuracy: 0.9314174652099609
